In [1]:
#pip install tensorflow==2.2.0

In [2]:
from loaders.Loaders import CSVLoader
from compoundFeaturization.rdkitFingerprints import MorganFingerprint
from splitters.splitters import SingletaskStratifiedSplitter

from sklearn.ensemble import RandomForestRegressor
from models.sklearnModels import SklearnModel
from metrics.Metrics import Metric
from metrics.metricsFunctions import r2_score, mean_absolute_error, mean_squared_error, median_absolute_error

import tensorflow as tf
print(tf.version.VERSION)

from parameterOptimization.HyperparameterOpt import HyperparamOpt_Valid, HyperparamOpt_CV


2.2.0


In [3]:
#Load Dataset
dataset = CSVLoader(dataset_path='data/PC-3.csv', 
                    mols_field='smiles', 
                    labels_fields='pIC50')
dataset = dataset.create_dataset()
dataset.get_shape()

Mols_shape:  4294
Features_shape:  X not defined!
Labels_shape:  (4294,)


In [4]:
#Featurization
dataset = MorganFingerprint().featurize(dataset)
dataset.get_shape()

Featurizing datapoint 0
Featurizing datapoint 1000
Featurizing datapoint 2000
Featurizing datapoint 3000
Featurizing datapoint 4000
Mols_shape:  4294
Features_shape:  (4294, 1024)
Labels_shape:  (4294,)


In [5]:
#Data Split
splitter = SingletaskStratifiedSplitter()
train_dataset, valid_dataset, test_dataset = splitter.train_valid_test_split(dataset=dataset, frac_train=0.6, 
                                                                             frac_valid=0.2, frac_test=0.2)

train_dataset.get_shape()
valid_dataset.get_shape()
test_dataset.get_shape()

Mols_shape:  2574
Features_shape:  (2574, 1024)
Labels_shape:  (2574,)
Mols_shape:  858
Features_shape:  (858, 1024)
Labels_shape:  (858,)
Mols_shape:  858
Features_shape:  (858, 1024)
Labels_shape:  (858,)


In [6]:
#Scikit-Learn Random Forest
#rf = RandomForestRegressor()
#model = SklearnModel(model=rf)

In [7]:
from models.kerasModels import KerasModel
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


input_dim = train_dataset.X.shape[1]
print(input_dim)

def build_model():
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=[1024]))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1))
    
    optimizer = tf.keras.optimizers.RMSprop(0.001)

    model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])
    return model

model = KerasModel(build_model, mode='regression', epochs = 5, verbose=1)

1024


In [8]:
#cross validation
model.cross_validate(dataset, Metric(r2_score), folds=3)

Computing K-fold split
Epoch 1/5
287/287 [==============================] - 0s 1ms/step - loss: 1.4856 - mae: 0.8477 - mse: 1.4856
Epoch 2/5
287/287 [==============================] - 0s 1ms/step - loss: 0.5172 - mae: 0.5391 - mse: 0.5172
Epoch 3/5
287/287 [==============================] - 0s 1ms/step - loss: 0.4014 - mae: 0.4729 - mse: 0.4014
Epoch 4/5
287/287 [==============================] - 0s 2ms/step - loss: 0.3166 - mae: 0.4179 - mse: 0.3166
Epoch 5/5
287/287 [==============================] - 0s 2ms/step - loss: 0.2599 - mae: 0.3812 - mse: 0.2599
Train Score: 
287/287 [==============================] - 0s 740us/step
r2_score: 
 0.8191759538261746
Test Score: 
144/144 [==============================] - 0s 789us/step
r2_score: 
 -0.2937644614269035
Epoch 1/5
287/287 [==============================] - 0s 2ms/step - loss: 1.8062 - mae: 0.9115 - mse: 1.8062
Epoch 2/5
287/287 [==============================] - 0s 2ms/step - loss: 0.5425 - mae: 0.5567 - mse: 0.5425
Epoch 3/5
287/287

(None,
 0,
 0,
 [0.8191759538261746, 0.857372969244566, 0.8278168940443085],
 [-0.2937644614269035, -0.12359089763843012, -0.34035854921703357],
 0.8347886057050165,
 -0.25257130276078904)

In [9]:
# model training
model.fit(train_dataset)

kerasModels fit!
Epoch 1/5
258/258 [==============================] - 0s 2ms/step - loss: 1.6251 - mae: 0.9090 - mse: 1.6251
Epoch 2/5
258/258 [==============================] - 0s 2ms/step - loss: 0.5984 - mae: 0.5843 - mse: 0.5984
Epoch 3/5
258/258 [==============================] - 0s 2ms/step - loss: 0.4122 - mae: 0.4865 - mse: 0.4122
Epoch 4/5
258/258 [==============================] - 0s 2ms/step - loss: 0.3325 - mae: 0.4310 - mse: 0.3325
Epoch 5/5
258/258 [==============================] - 0s 1ms/step - loss: 0.2597 - mae: 0.3816 - mse: 0.2597


In [10]:
metrics = [Metric(mean_absolute_error), Metric(mean_squared_error), Metric(median_absolute_error), Metric(r2_score)]
print("#############################")
# evaluate the model
print('Training Dataset: ')
train_score = model.evaluate(train_dataset, metrics)
print("#############################")
print('Validation Dataset: ')
valid_score = model.evaluate(valid_dataset, metrics)
print("#############################")
print('Test Dataset: ')
test_score = model.evaluate(test_dataset, metrics)


#############################
Training Dataset: 
<class 'tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor'>
258/258 [==============================] - 0s 780us/step
mean_absolute_error: 
 0.40621601859543643
mean_squared_error: 
 0.2661763305951256
median_absolute_error: 
 0.3411395931243897
r2_score: 
 0.7482459082130138
#############################
Validation Dataset: 
<class 'tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor'>
86/86 [==============================] - 0s 774us/step
mean_absolute_error: 
 0.5694194081273594
mean_squared_error: 
 0.5534601176673768
median_absolute_error: 
 0.459528036117554
r2_score: 
 0.47896642995201577
#############################
Test Dataset: 
<class 'tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor'>
86/86 [==============================] - 0s 782us/step
mean_absolute_error: 
 0.576172646432868
mean_squared_error: 
 0.5455793063416362
median_absolute_error: 
 0.48017868041992173
r2_score: 
 0.4853250197669864


In [11]:
#Build a model function for hyperparameter optimization
def rf_model_builder(n_estimators=10, max_features='auto', criterion='mse'):
    rf_model = RandomForestRegressor(n_estimators=n_estimators, max_features=max_features,
                                     criterion=criterion)
    return rf_model

params_dict_rf = {"n_estimators": [10, 100],
                  "max_features": ["auto", "sqrt", "log2", None],
                  "criterion": ["mse", "mae"]
                  }
    
model = SklearnModel(rf_model_builder, 'regression')

In [12]:
#Hyperparameter Optimization
#optimizer = HyperparamOpt_Valid(rf_model_builder)

#best_rf, best_hyperparams, all_results = optimizer.hyperparam_search(params_dict_rf, 
#                                                                     train_dataset, 
#                                                                     valid_dataset, 
#                                                                     Metric(r2_score),
#                                                                     n_iter_search=15)

#print('#################')
#print(best_hyperparams)
#print(best_rf)

In [13]:
#Evaluate model
#best_rf.evaluate(test_dataset, metrics)

In [ ]:
#Hyperparameter Optimization with CV
optimizer = HyperparamOpt_CV(rf_model_builder)

best_rf, best_hyperparams, all_results = optimizer.hyperparam_search('sklearn',
                                                                     params_dict_rf, 
                                                                     train_dataset,  
                                                                     'r2', 
                                                                     cv=3,
                                                                     n_iter_search=10,
                                                                    n_jobs=8)

print('#################')
print(best_hyperparams)
print(best_rf)

#Evaluate model
best_rf.evaluate(test_dataset, metrics)

MODEL TYPE:  sklearn
asdasdas
Fitting 10 random models from a space of 16 possible models.
